# Blue Brain BioExplorer
![](../bioexplorer_banner.png)

In [1]:
from bioexplorer import BioExplorer, Protein, Surfactant, Membrane, Volume, Virus,\
                        RNASequence, Cell, Sugars, Vector2, Vector3, Quaternion
import nglview

be = BioExplorer('localhost:5000')

Failed to display Jupyter Widget of type _ColormakerRegistry . 
 
 If you're reading this message in the Jupyter Notebook or JupyterLab Notebook, it may mean
 that the widgets JavaScript is still loading. If this message persists, it
 likely means that the widgets JavaScript library is either not installed or
 not enabled. See the Jupyter
 Widgets Documentation for setup instructions.
 
 
 If you're reading this message in another frontend (for example, a static
 rendering on GitHub or NBViewer ),
 it may mean that your frontend doesn't currently support widgets.

In [2]:
status = be.reset()

### Resources

In [3]:
resource_folder = '../../tests/test_files/'
pdb_folder = resource_folder + 'pdb/'
rna_folder = resource_folder + 'rna/'
glycan_folder = pdb_folder + 'glycans/'

glucose_path = pdb_folder + 'glucose.pdb'
lactoferrin_path=pdb_folder + 'immune/1b0l.pdb'
defensin_path = pdb_folder + 'immune/1ijv.pdb'

complex_paths = [glycan_folder + 'complex/33.pdb', glycan_folder + 'complex/34.pdb',
                 glycan_folder + 'complex/35.pdb',glycan_folder + 'complex/36.pdb']
high_mannose_paths = [glycan_folder + 'high-mannose/1.pdb', 
                      glycan_folder + 'high-mannose/2.pdb',
                      glycan_folder + 'high-mannose/3.pdb',
                      glycan_folder + 'high-mannose/4.pdb']
hybrid_paths = [glycan_folder + 'hybrid/24.pdb']
o_glycan_paths = [glycan_folder + 'o-glycan/12.pdb']

### Configuration

In [4]:
# Scene
scene_size = 250.0

# Proteins
protein_radius_multiplier = 1.0
protein_representation = be.REPRESENTATION_ATOMS# _AND_STICKS
glycan_radius_multiplier = 1.0
glycan_representation = be.REPRESENTATION_ATOMS_AND_STICKS
protein_load_hydrogen = False

# Viruses
nb_protein_s = 62
nb_protein_e = 42
nb_protein_m = 50
add_rna = True
add_glycans = False

## Camera

In [61]:
status = be.core_api().set_camera(
    orientation=[-0.008, 0.391, -0.084, 0.916],
    position=[11.029, -67.295, 35.860],
    target=[-27.555, -64.584, -1.414]
)

## Scene description

### Coronavirus

In [28]:
'''Get receptor protein instance transformation'''
instance_index = 3
transformation = be.get_protein_instance_transformation(
    assembly_name=name,
    name=name + '_' + be.NAME_RECEPTOR,
    instance_index=instance_index
)
print('Position = ' + str(transformation['position']))
print('Rotation = ' + str(transformation['rotation']))

Position = -35.5,-77.0984,-5
Rotation = 0.00180877,0.00242816,0.801949,0.597385


In [44]:
be.add_coronavirus(
    name='Coronavirus', resource_folder=resource_folder,
    representation=be.REPRESENTATION_ATOMS,
    position=Vector3(-35, -7.0, 13.0), add_glycans=True
)

### Host cell

In [53]:
# ACE2 receptor definition
ace2_receptor = Protein(
    sources=[pdb_folder + '6m18.pdb'],
    occurences=5,
    position=Vector3(0.0, 6.0, 0.0),
)

# Membrane definition
membrane_size = scene_size
membrane_height = 80.0
membrane = Membrane(
    sources=[pdb_folder + 'membrane/popc.pdb'],
    occurences=20000
)

# Cell definition
name='Cell'
cell = Cell(
    name=name,
    size=membrane_size, extra_parameters=[membrane_height],
    shape=be.ASSEMBLY_SHAPE_SINUSOIDAL,
    random_position_seed=1, random_position_strength=0.025,
    random_rotation_seed=2, random_rotation_strength=4.0,
    membrane=membrane, receptor=ace2_receptor)

# Add cell to scene
status = be.add_cell(
    cell=cell, representation=protein_representation,
    atom_radius_multiplier=protein_radius_multiplier,
    random_seed=1,
    position=Vector3(-7, -82.5, -3.0)
)

status = be.set_protein_color_scheme(
    name, name + '_' + be.NAME_RECEPTOR, 
    be.COLOR_SCHEME_CHAINS, 'OrRd_r', 7)

In [57]:
be.add_multiple_glycans(
    rotation=Quaternion(0.707, 0.0, 0.0, 0.707),
    representation=glycan_representation, assembly_name=name, 
    glycan_type=be.NAME_GLYCAN_COMPLEX,
    protein_name=be.NAME_RECEPTOR, paths=complex_paths, 
    indices=[53, 90, 103, 322, 432, 690])

be.add_multiple_glycans(
    rotation=Quaternion(0.707, 0.0, 0.0, 0.707),
    representation=glycan_representation, assembly_name=name,
    glycan_type=be.NAME_GLYCAN_HYBRID,
    protein_name=be.NAME_RECEPTOR, paths=hybrid_paths, 
    indices=[546])

indices = [[155, Quaternion(0.0, 0.707, 0.0, 0.707)],
           [730, Quaternion(0.0, 0.707, 0.0, 0.707)]]
for index in indices:
    o_glycan_name = name + '_' + be.NAME_GLYCAN_O_GLYCAN + '_' + str(index[0])
    o_glycan = Sugars(
        assembly_name=name, name=o_glycan_name, 
        source=o_glycan_paths[0],
        protein_name=name + '_' + be.NAME_RECEPTOR, 
        representation=glycan_representation,
        chain_ids=[2, 4], site_indices=[index[0]], 
        rotation=index[1])
    be.add_sugars(o_glycan)

## Materials

In [58]:
be.apply_default_color_scheme(
    shading_mode=be.SHADING_MODE_PERLIN,
    user_parameter=0.03, specular_exponent=5, glossiness=0.1)

IntProgress(value=0, max=30)

## Functional regions

### Spike

In [55]:
# RGB color palette for amino acid indices
grey = [0.2, 0.2, 0.2]
dark_green = [0.0, 1.0, 0.0]
light_green = [0.0, 1.0, 0.0]
red = [1.0, 0.0, 0.0]
green = [0.0, 1.0, 0.0]
blue = [0.0, 0.0, 1.0]
yellow = [1.0, 1.0, 0.0]
cyan = [1.0, 0.0, 1.0]

region_indices_and_colors = [
    [   1, grey ], [   16, grey], [  306, grey], [  330, grey], [438, dark_green], 
    [ 507, green], [  522, grey], [  816, grey ], [  835, grey ], [908, grey],
    [ 986, grey ], [ 1076, grey], [ 1274, grey], [ 2000, grey ]
]

# Build a palette according to region colors
palette = list()
for index in range(len(region_indices_and_colors)-1):
    for i in range(region_indices_and_colors[index + 1][0] - 
                   region_indices_and_colors[index][0]):
        palette.append(region_indices_and_colors[index][1])

# Apply palette to other chains
status = be.set_protein_color_scheme(
    assembly_name='Coronavirus', name='Coronavirus_' + be.NAME_PROTEIN_S_OPEN, 
    color_scheme=be.COLOR_SCHEME_CHAINS,
    palette_name='Greys_r', palette_size=100)

# Apply palette to region color scheme, only for chain #2
status = be.set_protein_color_scheme(
    assembly_name='Coronavirus', name='Coronavirus_' + be.NAME_PROTEIN_S_OPEN, 
    color_scheme=be.COLOR_SCHEME_REGION, palette=palette,
    chain_ids=[2]
)

### Receptor

In [56]:
# RGB color palette for amino acid indices
grey = [0.5, 0.5, 0.5]
dark_blue = [0.0, 0.0, 0.5]
light_blue = [0.0, 0.0, 1.0]

region_indices_and_colors = [
    [   1, grey      ], [   5, grey],  [  30, light_blue],  [  41, dark_blue],
    [  82, light_blue], [  84, grey],  [ 353, grey      ],  [ 357, grey     ],
    [ 800, grey      ]]

# Build a palette according to region colors
palette = list()
for index in range(len(region_indices_and_colors)-1):
    for i in range(region_indices_and_colors[index + 1][0] - 
                   region_indices_and_colors[index][0]):
        palette.append(region_indices_and_colors[index][1])

# Apply palette to other chains
name = 'Cell'
status = be.set_protein_color_scheme(
    assembly_name=name, name=name + '_' + be.NAME_RECEPTOR, 
    color_scheme=be.COLOR_SCHEME_CHAINS,
    palette_name='Greys_r', palette_size=100)

# Apply palette to region color scheme, only for chain #2
status = be.set_protein_color_scheme(
    assembly_name=name, name=name + '_' + be.NAME_RECEPTOR, 
    color_scheme=be.COLOR_SCHEME_REGION, palette=palette,
    chain_ids=[4])

## Rendering settings

In [47]:
status = be.core_api().set_renderer(
    background_color=[96 / 255, 125 / 255, 139 / 255],
    current='bio_explorer',
    samples_per_pixel=1, subsampling=4, max_accum_frames=64)
params = be.core_api().BioExplorerRendererParams()
params.gi_samples = 1
params.gi_weight = 0.4
params.gi_distance = 500
params.shadows = 1.0
params.soft_shadows = 1.0
params.fog_start = 1500
params.fog_thickness = 1500
params.max_bounces = 1
status = be.core_api().set_renderer_params(params)

## Snapshots

In [15]:
output_folder = '/tmp/'

In [16]:
from braynsmediamaker import MovieMaker
mm = MovieMaker(be.core_api())
mm.create_snapshot(
    path=output_folder + 'protein_s_and_ace2_receptor.png',
    size=[2048,2048], samples_per_pixel=64)

IntProgress(value=0, description='In progress...')

KeyboardInterrupt: 